# Uganda Panel: Create the Data Set



In [66]:
##################
##### Package
#################

###--- Data Managment
  library(readstata13)
  library(plyr)
  library(dplyr)
  library(tidyr)
  library(foreign) ## Export Data 
  
  ###--- OLS
  library(stargazer) # Get tables print
  library(lmtest)
  library(multiwayvcov)
  library(sandwich)

  
  ### Directories
  path_work <- "/Users/juancarlosmunoz/Box Sync/Uganda_LSMS/"
  graph_path <- "/Users/juancarlosmunoz/Box Sync/Uganda_LSMS/05_Paper/Graph/"
  save_data <-  "/Users/juancarlosmunoz/Box Sync/Uganda_LSMS/05_Analysis/"
  path_who <- "/Users/juancarlosmunoz/Box Sync/Uganda_LSMS/05_Analysis/inputs/"
  tables_path <- "/Users/juancarlosmunoz/Box Sync/Uganda_LSMS/06_Paper/tablets/"
  graph_path <- "/Users/juancarlosmunoz/Box Sync/Uganda_LSMS/06_Paper/Graph/"

# Loading Data

In [101]:
    #### 01 - Households Controls (ID - OK)
    load(paste0(save_data,"00_hhcontrols_Jun2018.Rda"))

  #### 03 - Agricultural Controls ()||@|… 
  load(paste0(save_data,"03_agr_controls_June2017.Rda"))

  #### 04 - Gross Agricultural Income
  load(paste0(save_data,"04_inc_gross_inc_Jun2018.Rda"))

    #### 05 - Expenditures
  load(paste0(save_data,"05_expenditure_Jun2018.Rda"))

    #### 05 - Expenditures
  load(paste0(save_data,"06_livestock_Jun2018.Rda"))


# Get the Share (descriptive)

In [6]:
### Test DataSet
f <- function(x) { ifelse(x>0,1,0)}
    
get_share <- function(inc_gross_agr_05) {inc_gross_agr_05 %>% ungroup(hhid) %>%  transmute(fruit=f(harv_q_fruit),cash=f(harv_q_cash)) %>% summarise(share_fruit=sum(fruit)/n(),share_cash=sum(cash)/n())}

get_share(inc_gross_agr_06)
get_share(inc_gross_agr_10)
get_share(inc_gross_agr_11)
get_share(inc_gross_agr_12)
get_share(inc_gross_agr_14)
    

share_fruit,share_cash
0.4938758,0.2611549


share_fruit,share_cash
0.2307033,0.2045455


share_fruit,share_cash
0.4983607,0.2398126


share_fruit,share_cash
0.4734252,0.2327756


share_fruit,share_cash
0.44158,0.2199584


# Create Function

In [38]:
##################
##### Build Data Sets
#################
  
    #### Function 2: Merge Data Set
  mer_data <- function(text,defl) {

    #### Variable
    ##-------- MAIN --
    data05_hh <- eval(parse(text =paste0("hhcontrols_",text))) %>% 
    ### Land
    left_join(eval(parse(text =paste0("agri_",text)))) %>% select(-hhid) %>%
    ## Expenditure 
    left_join(eval(parse(text =paste0("exp_",text)))) %>%
    ### Calorie intake
    #left_join(eval(parse(text =paste0("itk_kcal_",text)))) %>%
    ### Income Agriculture
    left_join(eval(parse(text =paste0("inc_gross_agr_",text)))) %>%
    ### Livestock
    left_join(eval(parse(text =paste0("lvst_",text)))) %>%
    ### Clean Data
    mutate_at(vars(sh_tof_w_b:lvst_poul),funs(((function(x){replace(x,is.na(x),0)})(.)))) %>%
    #### Deflacted data
      mutate_at(vars(exp_food:tot_exp,inc_fruit:inc_ag_total),funs(((function(x){x *defl})(.)))) %>%
    #### Main variblaes
      mutate(hhsize_ae=ifelse(hhsize_ae==0,hhsize,hhsize_ae)) %>%
    ### Expenditure in Foor Per Capita
      mutate(exp_food=exp_food_pur+exp_food_away+exp_food_own+exp_food_free) %>%
    #   mutate_at(vars(exp_food:tot_exp),funs(((function(x){x/hhsize_ae})(.)))) %>%
    ### Create the WAve Variable
      mutate(wave=text)
    
    return(data05_hh)
  }


In [105]:
dat06 <- mer_data("06",1.520308938) %>% select(-sh_timber)

### Fix dat14
hh_roster_14 <- read.dta13(paste0(path_work,"00_RawData/2013-14/GSEC1.dta"))
mer <- hh_roster_14 %>% transmute(HHID_old=HHID_old,hhid=HHID) %>% filter(!duplicated(hhid))
dat14 <- mer_data("14", 0.914632051) %>% mutate(comm=ea) %>% select(-HHID_05,-ea)

hh_roster_14 <-hh_roster_14 %>% left_join(mer) %>% transmute(HHID=as.character(HHID_old)) 

panel <- rbind(dat06,mer_data("10",1),mer_data("11",1.094444912),mer_data("12",0.991249995),dat14)


Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"
Warning message in read.dta13(paste0(path_work, "00_RawData/2013-14/GSEC1.dta")):
“
  urban:
  Duplicated factor levels detected - generating unique labels.
”Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID_old"
Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"
Warning message in `[<-.factor`(`*tmp*`, ri, value = c(13L, NA, 15L, NA, 51L, 17L, :
“invalid factor level, NA generated”Warning message in `[<-.factor`(`*tmp*`, ri, value = c(12L, NA, 16L, 15L, 17L, 17L, :
“invalid factor level, NA generated”

In [106]:
table(panel$wave)


  06   10   11   12   14 
3123 2975 2735 2850 3119 

In [102]:
dat14 <- mer_data("14", 0.914632051) %>% mutate(comm=ea) %>% select(-HHID_05,-ea)

length(dat14$HHID)

Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"
Joining, by = "HHID"


[1] 3119

In [108]:
### Panel data
write.csv(panel
          ,file=paste0(save_data,"Uganda_Panel_06282018.csv"))
  

ERROR: Error in write.dta(panel, file = paste0(save_data, "Uganda_Panel_06282018.dta")): empty string is not valid in Stata's documented format


In [ ]:
  ### Households Level Data Sets
  #data10_nutr <- nutr_10 %>% select(-pid_cg) %>% inner_join(data10_hh)
  #data11_nutr <- nutr_11 %>% select(-pid_cg) %>% inner_join(data11_hh)
  nutr_14$PID_Old[nchar(nutr_14$PID_Old)==0]="0"
  data14_nutr <- nutr_14 %>% inner_join(data14_hh)
  
  #### Save Data in Stata
  
  write.dta(data05_hh,paste0(save_data,"data05_hh.dta"))
  
  #write.dta(data10_hh,paste0(save_data,"data10_hh.dta"))
  #write.dta(data10_nutr,paste0(save_data,"data10_nutr.dta"))
  
  #write.dta(data11_hh,paste0(save_data,"data11_hh.dta"))
  #write.dta(data11_nutr,paste0(save_data,"data11_nutr.dta"))
  
  write.dta(data14_hh,paste0(save_data,"data14_hh.dta"))
  write.dta(data14_nutr,paste0(save_data,"data14_nutr.dta"))
  
  #save(data05_hh,data10_hh,data14_hh,data10_nutr,data14_nutr,file=paste0(save_data,"TreesOnFarm_Uganda_04012017.Rda"))
  
  save(data05_hh,data14_hh,file=paste0(save_data,"TreesOnFarm_Uganda_12012017.Rda"))

  